In [ ]:
import numpy as np

In [ ]:
%run data_prep.ipynb
df = df  # Janky way to import the df from data_prep
df

In [ ]:
print("DataFrame Info:")
df.info()

print("\nSummary Statistics:")
print(df.describe())

print("\nFirst 2 Rows:")
print(df.head(2))


# Works in progress

In [ ]:
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import pandas as pd

# Step 1: Adjust timestamps for game day
maggiesue_df = df[df['name'] == 'Maggiesue'].copy()
maggiesue_df['game_day'] = (maggiesue_df['timestamp'] - pd.Timedelta(hours=7)).dt.date  # Shift by 7 hours

# Step 2: Aggregate dungeon level data by game day
dungeon_level_columns = [col for col in maggiesue_df.columns if col.startswith('mplus_level_')]
grouped_df = maggiesue_df.groupby('game_day', as_index=False)[
    [*dungeon_level_columns, 'ilvl_high_watermark', 'm+_score']].max()

# Melt dungeon level data for stacking
stacked_grouped = grouped_df.melt(
    id_vars=['game_day', 'ilvl_high_watermark', 'm+_score'],
    value_vars=dungeon_level_columns,
    var_name='Dungeon Level',
    value_name='Dungeons Done'
)
stacked_grouped['Dungeon Level'] = stacked_grouped['Dungeon Level'].str.replace('mplus_level_', 'Level ')

# Create color mapping for dungeon levels
levels = sorted(stacked_grouped['Dungeon Level'].unique(), key=lambda x: int(x.split(' ')[1]))
colors = plt.cm.viridis(np.linspace(0, 1, len(levels)))
color_map = {level: mcolors.to_hex(color) for level, color in zip(levels, colors)}

# Step 3: Create the figure
fig = go.Figure()

# Add stacked bar traces for dungeon levels
for level in levels:
    level_data = stacked_grouped[stacked_grouped['Dungeon Level'] == level]
    fig.add_trace(
        go.Bar(
            x=level_data['game_day'],
            y=level_data['Dungeons Done'],
            name=level,
            marker=dict(color=color_map[level])
        )
    )

# Add the `ilvl_high_watermark` line
fig.add_trace(go.Scatter(
    x=grouped_df['game_day'],
    y=grouped_df['ilvl_high_watermark'],
    mode='lines',
    line=dict(color='red', width=2, dash='dot'),
    name='iLvl High Watermark',
    yaxis='y2'  # Use secondary y-axis
))

# Add the `m+_score` line
fig.add_trace(go.Scatter(
    x=grouped_df['game_day'],
    y=grouped_df['m+_score'],
    mode='lines',
    line=dict(color='green', width=2),
    name='M+ Score',
    yaxis='y3'  # Use tertiary y-axis
))

# Step 4: Update layout for multiple axes
fig.update_layout(
    barmode='stack',
    title="Maggiesue's Mythic+ Dungeon Levels Done (Aggregated by Game Day)",
    xaxis=dict(title='Game Day'),
    yaxis=dict(
        title='Number of Dungeons',
        side='left',
        # range=[0, stacked_grouped['Dungeons Done'].max() + 1],  # Adjust range for readability
    ),
    yaxis2=dict(
        title='',  # Remove the axis title
        overlaying='y',
        side='right',
        range=[580, 645],  # Range for ilvl_high_watermark
        showgrid=False,
        showticklabels=False  # Hide tick labels
    ),
    yaxis3=dict(
        title='',
        overlaying='y',
        side='right',
        position=1.0,  # Slightly offset to the right
        range=[0, 3200],  # Range for mplus_score
        showgrid=False,
        showticklabels=False  # Hide tick labels
    ),
    legend=dict(
        title="Legend",
        traceorder="reversed"  # Reverse legend order
    ),
    height=600
)

# Display the plot
fig.show()


# Used for site

In [ ]:
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

# Step 1: Group Data
data_grouped = df.groupby(['game_day', 'name'], as_index=False).agg(
    ilvl_high_watermark=('ilvl_high_watermark', 'max'),
    mplus_score=('m+_score', 'max'),
    cumulative_dungeons=('corrected_season_mythic_dungeons', 'max'),  # Use corrected cumulative column
    class_color=('class_color', 'first')  # Keep the class color
)

# Step 2: Create Full Player-Day Index
all_game_days = data_grouped["game_day"].unique()
all_players = data_grouped["name"].unique()
full_index = pd.MultiIndex.from_product([all_game_days, all_players], names=["game_day", "name"])
full_data = pd.DataFrame(index=full_index).reset_index()

# Step 3: Merge with Actual Data
data_grouped = full_data.merge(data_grouped, on=["game_day", "name"], how="left")

# Step 4: Forward-Fill Player Data to Preserve Ex-Players
data_grouped = data_grouped.sort_values(["name", "game_day"])
data_grouped["ilvl_high_watermark"] = data_grouped.groupby("name")["ilvl_high_watermark"].ffill()
data_grouped["mplus_score"] = data_grouped.groupby("name")["mplus_score"].ffill()
data_grouped["cumulative_dungeons"] = data_grouped.groupby("name")["cumulative_dungeons"].ffill()
data_grouped["class_color"] = data_grouped.groupby("name")["class_color"].ffill()

# Step 5: Fill Any Remaining NaNs with Defaults
data_grouped = data_grouped.fillna({
    "ilvl_high_watermark": 0,  # Assign a default iLvl
    "mplus_score": 0,  # Default M+ score
    "cumulative_dungeons": 0,  # Default dungeon count
    "class_color": "#888888"  # Assign a neutral color if missing
})

# Step 6: Create Color Mapping
name_to_color = dict(zip(data_grouped['name'], data_grouped['class_color']))

# Step 7: Create the Animated Bubble Chart
fig = px.scatter(
    data_grouped,
    x='ilvl_high_watermark',
    y='mplus_score',
    size='cumulative_dungeons',  # Bubble size as cumulative dungeons
    color='name',  # Use the player's name for color categories
    color_discrete_map=name_to_color,  # Map player names to their class colors
    animation_frame='game_day',
    category_orders={"name": sorted(df["name"].unique())},  # Ensures all players are included
    animation_group='name',  # Ensures continuity across frames
    title="Player Daily Progression Over the Season",
    labels={
        'ilvl_high_watermark': 'iLvl High Watermark',
        'mplus_score': 'Mythic+ Score',
        'cumulative_dungeons': 'Cumulative Dungeons Done',
        'game_day': 'Game Day',
        'name': 'Player'
    },
    hover_name='name',
)
fig.update_traces(marker=dict(sizemode='area', sizeref=data_grouped['cumulative_dungeons'].max() / 50))

# Step 8: Dynamic Axis Scaling
default_range_x = [max(data_grouped['ilvl_high_watermark'].min() - 1, 600),
                   data_grouped['ilvl_high_watermark'].max() + 1]
default_range_y = [max(data_grouped['mplus_score'].min() - 50, 0),
                   data_grouped['mplus_score'].max() + 50]

fig.update_layout(
    xaxis=dict(title='iLvl High Watermark', range=default_range_x),
    yaxis=dict(title='Mythic+ Score', range=default_range_y),
    showlegend=False,  # Hide the legend for simplicity
    height=600,
    legend_title="Player"
)

# Step 9: Adjust Frame Ranges Without Interpolated Animations
animated_range_x = default_range_x
animated_range_y = default_range_y
target_range_x = animated_range_x.copy()
target_range_y = animated_range_y.copy()
xd, yd = 0, 0
frame_num = 0

for frame in fig.frames:
    frame_num += 1
    # frame_date = pd.to_datetime(frame.name).date()
    frame_date = str(frame.name)
    filtered_data = data_grouped[data_grouped['game_day'] == frame_date]

    # Filter out zero values before computing min
    filtered_ilvl = filtered_data.loc[filtered_data['ilvl_high_watermark'] > 0, 'ilvl_high_watermark']
    filtered_mplus = filtered_data.loc[filtered_data['mplus_score'] > 0, 'mplus_score']

    # Compute min values while ignoring NaNs and zeros, with fallbacks
    min_ilvl = filtered_ilvl.min(skipna=True) if not filtered_ilvl.empty else None
    min_mplus = filtered_mplus.min(skipna=True) if not filtered_mplus.empty else None

    # Apply default values if all entries were NaN or zero
    minx = max(min_ilvl - 1 if min_ilvl is not None else 600, 600)
    miny = max(min_mplus - 50 if min_mplus is not None else 0, 0)

    if minx > target_range_x[0] + 1 and frame_num > 50:
        target_range_x[0] = minx
        xd = (target_range_x[0] - animated_range_x[0]) / 5
    if miny > target_range_y[0] + 50 and frame_num > 50:
        target_range_y[0] = miny
        yd = (target_range_y[0] - animated_range_y[0]) / 5

    if animated_range_x[0] < target_range_x[0]:
        animated_range_x[0] = min(animated_range_x[0] + xd, target_range_x[0])
    if animated_range_y[0] < target_range_y[0]:
        animated_range_y[0] = min(animated_range_y[0] + yd, target_range_y[0])

    frame.layout = go.Layout(
        xaxis=dict(range=animated_range_x, autorange=False),
        yaxis=dict(range=animated_range_y, autorange=False),
    )

# Step 10: Disable Interpolated Animation During Playback
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            showactive=False,
            buttons=[
                dict(label="Play",
                     method="animate",
                     args=[None, {
                         "frame": {"duration": 100, "redraw": True},
                         "transition": {"duration": 60},
                         "mode": "immediate"  # No smooth transitions
                     }]),
                dict(label="Stop",
                     method="animate",
                     args=[[None], {
                         "frame": {"duration": 0, "redraw": False},
                         "mode": "immediate"
                     }])
            ]
        )
    ],
)

# Step 11: Show and Save the Plot
fig.show()

fig.write_html(
    "../../QuartzTWWS1/static/S1_ilvl_m+score_m+done.html",
    full_html=True,
    include_plotlyjs="cdn",
    auto_play=False
)


In [ ]:
import plotly.express as px
import pandas as pd

# Step 1: Prepare the data
data = df.copy()
data['game_day'] = (data['timestamp'] - pd.Timedelta(hours=7)).dt.date  # Adjust for game day

# Aggregate iLvl data by game day and player
data_grouped = data.groupby(['game_day', 'name'], as_index=False).agg(
    ilvl_high_watermark=('ilvl_high_watermark', 'max'),  # Track max iLvl
    class_color=('class_color', 'first')  # Keep the class color
)

# Step 2: Get all unique game days and players
all_game_days = data_grouped["game_day"].unique()
all_players = data_grouped["name"].unique()

# Step 3: Create a full index of every (game_day, name) combination
full_index = pd.MultiIndex.from_product([all_game_days, all_players], names=["game_day", "name"])
full_data = pd.DataFrame(index=full_index).reset_index()

# Step 4: Merge with the actual data, initially filling missing values with NaNs
data_grouped = full_data.merge(data_grouped, on=["game_day", "name"], how="left")

# Step 5: Forward-fill missing data for players who stopped playing
data_grouped = data_grouped.sort_values(["name", "game_day"])  # Ensure order before filling
data_grouped["ilvl_high_watermark"] = data_grouped.groupby("name")["ilvl_high_watermark"].ffill()
data_grouped["class_color"] = data_grouped.groupby("name")["class_color"].ffill()

# Step 6: Fill any remaining NaNs with default values (for newly introduced players)
data_grouped = data_grouped.fillna({
    "ilvl_high_watermark": 0,  # Assign a default iLvl for unknowns
    "class_color": "#888888"  # Neutral gray color for missing players
})

# Step 7: Ensure sorting is preserved for each game day
data_grouped = data_grouped.sort_values(["game_day", "ilvl_high_watermark"], ascending=[True, False])

# Create a name-to-color mapping
name_to_color = dict(zip(data_grouped['name'], data_grouped['class_color']))
print(name_to_color)

# Step 8: Create the animated bar chart race
fig = px.bar(
    data_grouped,
    x='ilvl_high_watermark',
    y='name',
    color='name',
    animation_frame='game_day',
    orientation='h',  # Horizontal bars
    title="iLvl Progression Race Over the Season",
    text='ilvl_high_watermark',  # Add iLvl value to the end of each bar
    labels={
        'ilvl_high_watermark': 'iLvl High Watermark',
        'name': 'Player',
        'game_day': 'Game Day',
    },
    color_discrete_map=name_to_color,  # Use WoW class colors
    category_orders={"name": sorted(all_players)},  # Force all names to be recognized
    animation_group="name",  # Keep continuity across frames
)

# Step 9: Update layout for better visuals
fig.update_layout(
    xaxis=dict(title='iLvl High Watermark', range=[575, 642]),  # Set iLvl range
    yaxis=dict(title='Player', categoryorder='total ascending'),  # Dynamic sorting
    height=800,
    legend_title="Class",
    showlegend=False,
)

# Step 10: Customize bar text appearance
fig.update_traces(
    textposition='outside',  # Display text at the end of each bar
    texttemplate='%{text:.2f}'  # Show iLvl as an integer
)

# Show the plot
fig.show()

# Save the plot to an HTML file
fig.write_html(
    "../../QuartzTWWS1/static/S1_ilvl_race.html",
    full_html=True,
    include_plotlyjs="cdn",
    auto_play=False  # Prevent auto-play on page load
)


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd

# Sort player names alphabetically
player_names = sorted(df['name'].unique())
n_players = len(player_names)

# Define grid size (adjust based on player count)
cols = min(3, n_players)  # Up to 3 columns per row
rows = int(np.ceil(n_players / cols))

# Create shared subplot figure
fig = make_subplots(
    rows=rows, cols=cols, shared_xaxes=True, shared_yaxes=True,
    subplot_titles=[f"<span style='color:{df[df['name'] == player]['class_color'].iloc[0]}'>{player}</span>" for player
                    in player_names],
    vertical_spacing=0.05, horizontal_spacing=0.05
)

# Create color mapping for dungeon levels
dungeon_level_columns = [col for col in df.columns if col.startswith('mplus_level_')]
levels = sorted([col.replace('mplus_level_', 'Level ') for col in dungeon_level_columns],
                key=lambda x: int(x.split(' ')[1]))
colors = plt.cm.viridis(np.linspace(0, 1, len(levels)))
color_map = {level: mcolors.to_hex(color) for level, color in zip(levels, colors)}

# Determine max dungeons done by any player on any game_day
max_dungeons_done = df[dungeon_level_columns].sum(axis=1).max()

# Loop through each player and add traces
for i, player in enumerate(player_names):
    row = (i // cols) + 1
    col = (i % cols) + 1

    # Filter data for the player
    player_df = df[df['name'] == player].copy()

    # Aggregate dungeon level data by game day
    grouped_df = player_df.groupby('game_day', as_index=False)[dungeon_level_columns].max()
    grouped_df['Total Dungeons Done'] = grouped_df[dungeon_level_columns].sum(axis=1)

    # Melt dungeon level data for stacking
    stacked_grouped = grouped_df.melt(
        id_vars=['game_day', 'Total Dungeons Done'],
        value_vars=dungeon_level_columns,
        var_name='Dungeon Level',
        value_name='Dungeons Done'
    )
    stacked_grouped['Dungeon Level'] = stacked_grouped['Dungeon Level'].str.replace('mplus_level_', 'Level ')

    # Ensure bars are showing by filtering non-zero values
    for level in levels:
        level_data = stacked_grouped[
            (stacked_grouped['Dungeon Level'] == level) & (stacked_grouped['Dungeons Done'] > 0)]
        if not level_data.empty:
            fig.add_trace(
                go.Bar(
                    x=level_data['game_day'],
                    y=level_data['Dungeons Done'],
                    name=level if i == 0 else None,  # Add legend only to the first subplot
                    marker=dict(color=color_map[level]),
                    showlegend=(i == 0),
                    hovertemplate=(
                        "Game Day: %{x}<br>"
                        "%{customdata[0]}: %{y}<br>"
                        "Weekly total: %{customdata[1]}<extra></extra>"
                    ),
                    customdata=np.stack((level_data['Dungeon Level'], level_data['Total Dungeons Done']), axis=-1)
                ),
                row=row, col=col
            )

    # Update layout for this specific subplot
    fig.update_yaxes(title_text='', range=[0, max_dungeons_done], row=row, col=col)

# Ensure x-axis labels appear on the correct bottom row only for missing columns
if n_players % cols != 0:
    missing_cols = cols - (n_players % cols)
    for col in range(cols - missing_cols + 1, cols + 1):
        fig.update_xaxes(showticklabels=True, row=rows - 1, col=col)

# We need a final update to the figure, to set its overall height
fig.update_layout(
    barmode='stack',
    title='Mythic+ Dungeon Levels Done by Player (Aggregated by Game Day)',
    height=rows * 300,
    width=cols * 500,
    showlegend=False,
)

# Save as HTML
fig.write_html("../../QuartzTWWS1/static/S1_mplus_player_grid.html", full_html=True, include_plotlyjs="cdn")

# Display
fig.show()


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import pandas as pd

# Ensure 'game_day' is a datetime type
df["game_day"] = pd.to_datetime(df["game_day"])

# Sort player names alphabetically
player_names = sorted(df["name"].unique())
n_players = len(player_names)

# Define grid size
cols = min(3, n_players)  # Up to 3 columns per row
rows = int(np.ceil(n_players / cols))

# Create shared subplot figure
fig = make_subplots(
    rows=rows, cols=cols, shared_xaxes=True, shared_yaxes=True,
    subplot_titles=[f"<span style='color:{df[df['name'] == player]['class_color'].iloc[0]}'>{player}</span>" for player
                    in player_names],
    vertical_spacing=0.05, horizontal_spacing=0.05
)

# Crest colors for lines (using a readable color palette)
crest_colors = {
    "weathered_crests": "blue",
    "carved_crests": "orange",
    "runed_crests": "green",
    "gilded_crests": "red"
}

# Determine max crest count for uniform y-axis scaling
max_crests_earned = df[["weathered_crests", "carved_crests", "runed_crests", "gilded_crests"]].max().max()

# Loop through each player and add traces
for i, player in enumerate(player_names):
    row = (i // cols) + 1
    col = (i % cols) + 1

    # Filter data for the player
    player_df = df[df["name"] == player].copy()

    # Aggregate crest data by game day
    grouped_df = player_df.groupby("game_day", as_index=False)[
        ["weathered_crests", "carved_crests", "runed_crests", "gilded_crests"]
    ].max()

    # Add traces for each crest type
    for crest, color in crest_colors.items():
        fig.add_trace(
            go.Scatter(
                x=grouped_df["game_day"],
                y=grouped_df[crest],
                mode="lines+markers",
                name=crest.replace("_", " ").title() if i == 0 else None,  # Show legend only in the first subplot
                marker=dict(size=1),
                line=dict(color=color, width=2),
                showlegend=(i == 0),
                hovertemplate=f"Game Day: %{{x}}<br>{crest.replace('_', ' ').title()}: %{{y}}<extra></extra>"
            ),
            row=row, col=col
        )

    # Update layout for this specific subplot
    fig.update_yaxes(title_text="", range=[0, max_crests_earned], row=row, col=col)

# Ensure x-axis labels appear on the correct bottom row only for missing columns
if n_players % cols != 0:
    missing_cols = cols - (n_players % cols)
    for col in range(cols - missing_cols + 1, cols + 1):
        fig.update_xaxes(showticklabels=True, row=rows, col=col)

# Update the overall layout
fig.update_layout(
    title="Crests Earned by Player (Aggregated by Game Day)",
    height=rows * 300,
    width=cols * 500,
    template="plotly_dark",
    showlegend=True
)

# Save as HTML (optional)
fig.write_html("../../QuartzTWWS1/static/S1_crests_player_grid.html", full_html=True, include_plotlyjs="cdn")

# Display
fig.show()


In [ ]:
import plotly.express as px
import pandas as pd

# Define dungeon total columns
dungeon_total_columns = [
    "arakara_city_of_echoes_total",
    "city_of_threads_total", "the_stonevault_total",
    "the_dawnbreaker_total", "grim_batol_total", "mists_of_tirna_scithe_total",
    "siege_of_boralus_total", "the_necrotic_wake_total"
]

# Ensure 'timestamp' is a datetime type and get the latest entry per player
df["timestamp"] = pd.to_datetime(df["timestamp"])
latest_per_player = df.sort_values("timestamp").groupby("name").last()

# Sum up the latest dungeon totals across all players
dungeon_totals = latest_per_player[dungeon_total_columns].sum()

# Create a DataFrame for Plotly
dungeon_totals_df = pd.DataFrame({"Dungeon": dungeon_totals.index, "Total Runs": dungeon_totals.values})
dungeon_totals_df["Dungeon"] = dungeon_totals_df["Dungeon"].str.replace("_total", "").str.replace("_", " ").str.title()

# Create the bar chart
fig = px.bar(
    dungeon_totals_df,
    x="Dungeon",
    y="Total Runs",
    title="Total Dungeon Runs by All Team Members (Final Season Totals)",
    labels={"Total Runs": "Total Runs", "Dungeon": "Dungeon"},
    text="Total Runs",
    color="Total Runs",
    color_continuous_scale="viridis"
)

# Improve layout
fig.update_layout(
    xaxis_tickangle=-45,
    template="plotly_dark",
    coloraxis_showscale=False
)

# Show the figure
fig.show()

fig.write_html("../../QuartzTWWS1/static/S1_total_team_dungeons.html", full_html=True, include_plotlyjs="cdn")



In [ ]:
import plotly.express as px
import pandas as pd

# Define dungeon total columns
dungeon_total_columns = [
    "arakara_city_of_echoes_total", "city_of_threads_total", "the_stonevault_total",
    "the_dawnbreaker_total", "grim_batol_total", "mists_of_tirna_scithe_total",
    "siege_of_boralus_total", "the_necrotic_wake_total"
]

# Compute the max dungeon count per player per game day
max_per_player_per_day = df.groupby(["game_day", "name"])[dungeon_total_columns].max().reset_index()

# Sum up these max values per dungeon for each game day
daily_dungeon_totals_corrected = max_per_player_per_day.groupby("game_day")[dungeon_total_columns].sum().reset_index()

# Melt data for Plotly (long format for multiple lines)
running_totals_melted_corrected = daily_dungeon_totals_corrected.melt(id_vars=["game_day"],
                                                                      value_vars=dungeon_total_columns,
                                                                      var_name="Dungeon",
                                                                      value_name="Total Runs")

# Clean dungeon names
running_totals_melted_corrected["Dungeon"] = running_totals_melted_corrected["Dungeon"].str.replace("_total",
                                                                                                    "").str.replace("_",
                                                                                                                    " ").str.title()

# Create the line chart
fig = px.line(
    running_totals_melted_corrected,
    x="game_day",
    y="Total Runs",
    color="Dungeon",
    title="Cumulative Dungeon Runs Over the Season",
    labels={"game_day": "Game Day", "Total Runs": "Total Runs"},
    markers=False
)

# Improve layout
fig.update_layout(
    xaxis_tickangle=-45,
    template="plotly_dark"
)

# Show the figure
fig.show()

fig.write_html("../../QuartzTWWS1/static/S1_team_dungeons_by_day.html", full_html=True,
               include_plotlyjs="cdn")


In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# Ensure timestamp is a datetime type
df["timestamp"] = pd.to_datetime(df["timestamp"])

# Compute team-level statistics per day
team_stats = df.groupby("timestamp")["ilvl_high_watermark"].agg(["mean", "std"]).reset_index()
team_stats["+1σ"] = team_stats["mean"] + team_stats["std"]
team_stats["-1σ"] = team_stats["mean"] - team_stats["std"]
team_stats["+2σ"] = team_stats["mean"] + (2 * team_stats["std"])
team_stats["-2σ"] = team_stats["mean"] - (2 * team_stats["std"])

# Create the player-level dataset
player_ilvl = df[["timestamp", "name", "ilvl_high_watermark", "class_color"]]

# Create the figure
fig = go.Figure()

# Add shaded standard deviation bands

fig.add_traces([
    go.Scatter(
        x=team_stats["timestamp"], y=team_stats["+1σ"],
        fill=None, mode="lines", line=dict(color="rgba(0,0,0,0)"),  # Invisible upper boundary
        showlegend=False
    ),
    go.Scatter(
        x=team_stats["timestamp"], y=team_stats["-1σ"],
        fill="tonexty", mode="lines", fillcolor="rgba(50, 204, 57, 0.2)",  # Darker for ±1σ
        line=dict(color="rgba(0,0,0,0)"),
        name="±1σ"
    ),
    go.Scatter(
        x=team_stats["timestamp"], y=team_stats["+2σ"],
        fill=None, mode="lines", line=dict(color="rgba(0,0,0,0)"),  # Invisible upper boundary
        showlegend=False
    ),
    go.Scatter(
        x=team_stats["timestamp"], y=team_stats["-2σ"],
        fill="tonexty", mode="lines", fillcolor="rgba(240, 230, 110, 0.1)",  # Lighter for ±2σ
        line=dict(color="rgba(0,0,0,0)"),
        name="±2σ"
    ),
])

# Add team average line
fig.add_scatter(x=team_stats["timestamp"], y=team_stats["mean"], mode="lines",
                name="Team Average", line=dict(color="green", width=3))

# Sort player names alphabetically for the legend
sorted_players = sorted(player_ilvl["name"].unique())

# Add each player's item level progression with their class color
for player in sorted_players:
    player_data = player_ilvl[player_ilvl["name"] == player]
    player_color = player_data["class_color"].iloc[0]  # Get class color
    fig.add_trace(go.Scatter(
        x=player_data["timestamp"],
        y=player_data["ilvl_high_watermark"],
        mode="lines",
        name=player,
        line=dict(color=player_color)
    ))

# Set y-axis range from 580 to 645, adjust height, and sort legend order
fig.update_layout(
    yaxis=dict(range=[580, 645]),
    xaxis_tickangle=-45,
    template="plotly_dark",
    title="Item Level Progression Over the Season",
    xaxis_title="Date",
    yaxis_title="Item Level",
    height=800,
    legend_traceorder="normal"  # Puts Team Average and StdDevs at the top
)

# Show the figure
fig.show()

fig.write_html("../../QuartzTWWS1/static/S1_ilvl_season_lines.html", full_html=True, include_plotlyjs="cdn")

